In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.4 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from pathlib import Path

import pandas as pd
import spacy

f = Path(f"./tlg0525.tlg001.perseus-eng2.pickle")

def tokenize(df: pd.DataFrame):
    model = "en_core_web_sm"

    nlp = spacy.load(model, disable=["ner"])

    df["tokens"] = df["unannotated_strings"].apply(nlp.tokenizer)

    raw_texts = [t for t in df["unannotated_strings"]]
    annotated_texts = nlp.pipe(raw_texts, batch_size=100)

    df["nlp_docs"] = list(annotated_texts)

    return df

df = tokenize(pd.read_pickle(f))

# Bug 1

Why is the function `expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1)` returning `0.0` no matter what we pass to it?

In [ ]:
# Bugged code

from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """

    lemmata = [t for t in df['nlp_docs'].explode()]
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainlaind = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainlaind

In [ ]:
# Fixed code

from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """
    lemmata = [t.lemma_ for t in df['nlp_docs'].explode()]
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainland = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainland

3.2848751741271176

# Bug 1 analysis and response

### The bug occurs in the line "lemmata = [t for t in df['nlp_docs'].explode()]", where the current implementation lemmata does not account for the lemmatized words in  string format, but rather only accounts for the token objects. By using "t.lemma_ for t" instead of "t for t", we can effectively create the lemmata such that the counter correcly counts the frequency of the words using the lemmatized forms and string types, which would allow us to subsequently get the correct counts for the string of the node and collocate word that we pass as input.

# Bug 2

Why does the code below not give us the number of occurrences of the token "the"?

In [ ]:
# Bugged code

observed_freq_the = df['tokens'].str.find('the')

observed_freq_the

In [ ]:
# Fixed code

observed_freq_the = len([t for t in df["nlp_docs"].explode() if t.lemma_ == "the"])
observed_freq_the

<class 'list'>


# Bug 2 analysis and response

### In this case the bug occurs in the line "observed_freq_the = df['tokens'].str.find('the')", where it returns NaN because the data in the df["tokens"] are  token objects, but the str.find() operation intends for the data to be in string format. The line of code also would simply find the first entry of "the" in the tokens section of the dataframe, which would not give the number of occurences of "the." Therefore, the correct approach would be to employ a similar approach to Bug 1, where you would iterate through the tokens in the nlp docs and use t.lemma_ to check if the lemma of each token is "the," where it would then add it to the list. Finally, taking the length of the list would the total number of occurences.

# Bug 3

How is `o11` greater than `r1`, and why is our `o12` negative?

In [ ]:
# Bugged code

node = "mainland"
collocate = "the"

def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    # the right-hand side of a slice in Python is exclusive, so we add 1 to make sure
    # we're actually getting one element to the right

    chunked_lemmata = [lemmata[i - l_size:i + r_size + 1] for i in range(0, len(lemmata))]
    
    cooccurrences = [1 for l in chunked_lemmata if w1 in l and w2 in l]

    return sum(cooccurrences)

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")


In [ ]:
# Fixed code

node = "mainland"
collocate = "the"

def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]

    cooccurrences = 0

    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))
        window = lemmata[left:right]

        if w2 in window:
            cooccurrences += 1

    return cooccurrences

def count_ngram_non_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]

    non_cooccurrences = 0

    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))
        window = lemmata[left:right]

        if w2 not in window:
            non_cooccurrences += 1

    return non_cooccurrences


r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = df["nlp_docs"].apply(count_ngram_non_collocations, args=(node, collocate)).sum()
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 23, o12: 15, o21: 26909


# Bug 3 analysis and response

### The reason that o11 (number of cooccurences of the node and collocate) exceeds r1 (number of instances of the node) is because of how the window surrounding the node is calculated in the count_ngram_collocations function. In the bugged code, it leads to overcounting of coocccurences, where there are potential overlaps of the windows that would leave to collocates being accounted for more than once. In the refined count_ngram_collocations, the "indexes" list in the fixed code correctly stores the index of each node,and the function subsequently creates a window for each node to ensure that each instance of a collocate is correctly counted uniquely. Furthermore, the fact that o12 (number of non cooccurences) is negative is attested to this error, where since o12 = r1 - o11, and o11 currently exceeds r1. To recalculate the number of non cooccurrences, the implementation of a count_ngram_non_collocations function helps explicitly count the instances where the collocate is not present in a method similar to count_ngram_collocations. This calculation is then used to create our o12 value in a method that mirrors our o11 calculation.